In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import os

In [2]:
## Loading data - an excell file :

data1 = pd.ExcelFile('Tranzact2.xlsx')
tranzact_data1 = data1.parse(0)

lamba = len(tranzact_data1)
tranzact_data1.columns

Index(['Company', 'Supplier', 'Item', 'Category', 'UOM', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16'],
      dtype='object')

In [3]:
tranzact_data2 = tranzact_data1.iloc[0:lamba, 0:3]    ## Remove unnecessary columns
tranzact_data2['Item'] = tranzact_data2['Item'].map(lambda x: x if type(x)!=str else x.lower()) ## make Items in lower case
tranzact_data = tranzact_data2.dropna() ## remove rows with cell value none
tranzact_data = tranzact_data.reset_index(drop=True)
tranzact_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28921 entries, 0 to 28920
Data columns (total 3 columns):
Company     28921 non-null object
Supplier    28921 non-null object
Item        28921 non-null object
dtypes: object(3)
memory usage: 677.9+ KB


In [4]:
## Cleaning the Item Description to create a feature matrix :

pd.options.mode.chained_assignment = None
print(tranzact_data['Item'][9938])

tranzact_data['features'] = tranzact_data['Item'].str.replace(r'\b\d+\b','')  ## remove integers
print(tranzact_data['features'][9938])

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\W',' ')   ## remove puntuations
print(tranzact_data['features'][9938])

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only
print(tranzact_data['features'][9938])

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\w\b','') ## remove stand alone single letters
print(tranzact_data['features'][9938])

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\s+',' ') ## remove gaps between words to singe gap
print(tranzact_data['features'][9938])


## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")

words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')

tranzact_data['features']= tranzact_data['features'].apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())
print(tranzact_data['features'][5])


4)
)
 
 
 
 
ms sheet


In [6]:
## Taking out the new file:

writer = pd.ExcelWriter('Tranzact2_Features.xlsx', engine='xlsxwriter')

tranzact_data.to_excel(writer, sheet_name='Sheet1')

writer.save()

In [5]:
## To check no. of Suppliers & no. of unique featuresin the data base :

print('Supplier = ', len(tranzact_data.Supplier.value_counts()))
print('Unique Features = ', len(tranzact_data.features.value_counts()))

Supplier =  2337
Unique Features =  16728


In [7]:
## Above indicates that there are 2337 suppliers supplying 28921 items(as per no. of rows)
## above indicates that there are 16728 unique features out of 28921 items(as per rows)

In [8]:
## Let's reduce no. of Suppliers : criterian - apperaing at least 10 times in the data frame

# tranzact_data_short = tranzact_data.groupby("Supplier").filter(lambda x: len(x) > 10)

# print(len(tranzact_data_short))
# print(len(tranzact_data_short.Supplier.value_counts()))

In [9]:
## Above indicates that there are now 391 suppliers supplying 23649 items(as per no. of rows)

In [195]:
tranzact_data.features[0] + ' ' + tranzact_data.features[1]

'zincoplast zinc zincoplast pyn'

In [6]:
## Removing duplicate features + Supplier combinations :

for i in range (len(tranzact_data)):
    
    idx_feature = []
    idx_Supplier = []
    idx_common = []

    idx_feature = tranzact_data.index[tranzact_data['features'] == tranzact_data['features'][i]].tolist()
    idx_Supplier = tranzact_data.index[tranzact_data['Supplier'] == tranzact_data['Supplier'][i]].tolist()
    idx_common = list(set(idx_feature).intersection(idx_Supplier))
    
    if len(idx_common) > 1:
        
        tranzact_data['Supplier'][idx_common[1:]] = np.nan
        
tranzact_data_final = tranzact_data.dropna()
tranzact_data_final = tranzact_data_final.reset_index(drop=True) 
        
# tranzact_data = tranzact_data.drop(tranzact_data.index[idx_common[j]][1:])
                   

In [7]:
## To cross check no. of Suppliers & no. of unique features in the data base :

print('Supplier = ', len(tranzact_data_final.Supplier.value_counts()))
print('Unique Features = ', len(tranzact_data_final.features.value_counts()))
print('Total no. of entrys = ', len(tranzact_data_final))
tranzact_data_final.head()

Supplier =  2337
Unique Features =  16728
Total no. of entrys =  19764


,Company,Supplier,Item,features
0,M/S Plastech,Pukhraj Additives LLP,zincoplast zinc,zincoplast zinc
1,M/S Plastech,Pukhraj Additives LLP,zincoplast pyn,zincoplast pyn
2,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup...",chromium manganese spring steel rolled sup9 wi...
3,Modern Engineering & Spring Company,Jay Khodiyar Steel & Metals,ms sheet 8 ft x 4 ft x 1.00 mm - 4 nos,ms sheet
4,Modern Engineering & Spring Company,Dynachem Engineers,ms plate 160 x 160 x 6 - 4 holes drg. z-413146...,ms plate holes drg rev ae0


In [8]:
# tranzact_data_final = tranzact_data_final[pd.notnull(tranzact_data_final['features'])]
print(len(tranzact_data_final))

for i in range(19764):
    if tranzact_data_final['features'][i] == '':
        tranzact_data_final['Supplier'][i] = np.nan
        
tranzact_data_final = tranzact_data_final.dropna()
print(len(tranzact_data_final))

19764
19761


In [214]:
group = tranzact_data_final.groupby('features')


required_frame = group.apply(lambda x: x['Supplier'].unique())


required_frame = required_frame.apply(pd.Series)


In [215]:
required_frame

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
features,,,,,,,,,,,,,,,,,,
,WAAREE INDUSTRIES PVT. LTD.,Vashi Electricals Pvt. Ltd,Ansar Wood Packing Service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00x12 tyre tube flap part,Trans-Lift,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
04cx6mm2 aluminium armoured cable,A.K.ELECTRICALS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
05h lm drill insert,Waaree Industries Pvt.Ltd.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0kui10stacmevtx insert vardex rh,Waaree Industries Pvt.Ltd.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0ohm,Vallabh Impex (India) Pvt. Ltd.,SMD Electro Components,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000pf 250v y5v pin,VALIANT ELECTRO SALES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000uf 25v pin,SUNITA SALES AGENCIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100e 100mw 100ppm,Vallabh Impex (India) Pvt. Ltd.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [216]:
writer1 = pd.ExcelWriter('Tranzact2_REQ_LIST.xlsx', engine='xlsxwriter')

required_frame.to_excel(writer1, sheet_name='Sheet1')

writer1.save()

In [9]:
col = ['Supplier', 'features']
tranzact_data_search = tranzact_data_final[col]

# tranzact_data_final.columns

tranzact_data_search['Supplier_id'] = tranzact_data_search['Supplier'].factorize()[0]

supplier_id_tranzact_data_search = tranzact_data_search[['Supplier', 'Supplier_id']].drop_duplicates(). \
                                                        sort_values('Supplier_id')

## Create two dictionaries :

supplier_to_id = dict(supplier_id_tranzact_data_search.values)
id_to_supplier = dict(supplier_id_tranzact_data_search[['Supplier_id', 'Supplier']].values)


# df.columns = ['Product', 'Consumer_complaint_narrative']
# df['category_id'] = df['Product'].factorize()[0]
# category_id_df = df[['Product', 'category_id']].drop_duplicates().sort_values('category_id')
# category_to_id = dict(category_id_df.values)
# id_to_category = dict(category_id_df[['category_id', 'Product']].values)
# df.head()


In [10]:

print(len(supplier_id_tranzact_data_search))
supplier_id_tranzact_data_search.head()

2337


,Supplier,Supplier_id
0,Pukhraj Additives LLP,0
2,Bijal Steels,1
3,Jay Khodiyar Steel & Metals,2
4,Dynachem Engineers,3
6,Sharp Engineering,4


In [11]:
tranzact_data_search.head()

,Supplier,features,Supplier_id
0,Pukhraj Additives LLP,zincoplast zinc,0
1,Pukhraj Additives LLP,zincoplast pyn,0
2,Bijal Steels,chromium manganese spring steel rolled sup9 wi...,1
3,Jay Khodiyar Steel & Metals,ms sheet,2
4,Dynachem Engineers,ms plate holes drg rev ae0,3


In [88]:
# Digitising Process No. 1

from sklearn.feature_extraction.text import TfidfVectorizer

# create the transform
my_vectorizer = TfidfVectorizer()

# tokenize and build vocab
digital_features = my_vectorizer.fit(tranzact_data_search['features'])
# summarize
# print(my_vectorizer.vocabulary_)

# encode document
my_vector = digital_features.transform(tranzact_data_search['features']).toarray()
# summarize encoded vector
print(my_vector.shape)

print(my_vector)

print(max(my_vector[0]))

(19761, 13421)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0.8110634360883946


In [13]:
## Digitising Process No. 2 : same as above but slightly different syntax

from sklearn.feature_extraction.text import TfidfVectorizer

#tfidf = TfidfVectorizer(sublinear_tf=True, min_df=2, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
#features = tfidf.fit_transform(df.Consumer_complaint_narrative).toarray()

#features.shape

my_tfidf = TfidfVectorizer()
digit_features = my_tfidf.fit_transform(tranzact_data_search.features).toarray()

print(digit_features.shape)

print(digit_features)


(19761, 13421)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## 
* sublinear_df is set to True to use a logarithmic form for frequency.
* min_df is the minimum numbers of documents a word must be present in to be kept.
* norm is set to l2, to ensure all our feature vectors have a euclidian norm of 1.
* ngram_range is set to (1, 2) to indicate that we want to consider both unigrams and bigrams.
* stop_words is set to "english" to remove all common pronouns ("a", "the", ...) to reduce the number of noisy * * * features.


In [14]:
## Digitising Process No. 3

my_my_tfidf = TfidfVectorizer(sublinear_tf=True, min_df=2, norm='l2', encoding='latin-1', \
                        ngram_range=(1, 2), stop_words='english')
dig_features = my_my_tfidf.fit_transform(tranzact_data_search.features).toarray()

print(dig_features.shape)
print(dig_features)


(19761, 16851)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:
## Digitising Process No. 4

from sklearn.feature_extraction.text import HashingVectorizer

y_my_vectorizer = HashingVectorizer(n_features=50)

y_my_vector = y_my_vectorizer.transform(tranzact_data_search['features'])

print(y_my_vector.shape)
#print((y_my_vector.toarray())[300:305,:])
print(y_my_vector.toarray())



(19761, 50)
[[ 0.          0.70710678  0.         ...  0.          0.
   0.        ]
 [ 0.          0.70710678  0.         ...  0.          0.
   0.        ]
 [ 0.          0.         -0.31622777 ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.         -0.4472136  ...  0.          0.
   0.        ]]


In [104]:
from math import*
 
def square_rooted(x):
 
    return round(sqrt(sum([a*a for a in x])),3)
 
def cosine_similarity(x,y):
 
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return round(numerator/float(denominator),3)

In [118]:
text = ['zincoplast zinc']
#text = ['subrata sen']

text_features = my_vectorizer.fit(text)

# encode document
text_vector = text_features.transform(text).toarray()

print(text_vector)
print((text_vector.shape)[1])

t_v_reshape = text_vector.reshape((text_vector.shape)[1],)

print(cosine_similarity(my_vector[0, :], t_v))
print(max(my_vector[0, :]))


[[0.70710678 0.70710678]]
2
0.0
0.8110634360883946


In [119]:
from sklearn.metrics.pairwise import cosine_similarity

# print(cosine_similarity([[1, 0, -1, 0]], [[-1,-1, 0]]))

# print(my_vector[0])

# cosine_similarity([my_vector[0]], [text_vector])
